<a href="https://colab.research.google.com/github/Arjitm26/Disease-Prediction-based-on-symptoms/blob/main/Disease_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score

In [ ]:
df = pd.read_csv('/kaggle/input/disease-symptoms-and-patient-profile-dataset/Disease_symptom_and_patient_profile_dataset.csv')

In [ ]:
df.head

In [ ]:
df.duplicated().sum()

In [ ]:
df.head()

In [ ]:
# Bar plot for Gender
sns.countplot(x='Gender', data=df)
plt.title('Distribution of Gender')
plt.show()

In [ ]:
sns.catplot(x = 'Outcome Variable' , y = 'Age' , data = df , kind = "swarm")

In [ ]:
sns.displot(df['Age'] , kde=True)

In [ ]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split
X = df.drop(['Outcome Variable'], axis=1)
y = df['Outcome Variable']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
# Initialize StandardScaler

In [ ]:

# Splitting into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the feature categories
binary_cols = ['Fever', 'Cough', 'Fatigue', 'Difficulty Breathing']
categorical_cols = ['Blood Pressure', 'Cholesterol Level', 'Gender']
numerical_cols = ['Age']

# Define transformers
ordinal_transformer = OrdinalEncoder(categories=[['No', 'Yes']] * len(binary_cols))
onehot_transformer = OneHotEncoder(drop='first', sparse=False)
scaler = StandardScaler()

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('binary', ordinal_transformer, binary_cols),
        ('categorical', onehot_transformer, categorical_cols),
        ('scaler', scaler, numerical_cols)
    ]
)


In [ ]:
onehot_encoder = OneHotEncoder(sparse=False)

In [ ]:
X_train_transformed = preprocessor.fit_transform(X_train)
x_test_transformed = preprocessor.transform(X_test)

In [ ]:
y_train_reshaped = np.array(y_train).reshape(-1, 1)
y_test_reshaped = np.array(y_test).reshape(-1, 1)

# Fit and transform the training data
y_train_encoded = onehot_encoder.fit_transform(y_train_reshaped)

# Transform the test data
y_test_encoded = onehot_encoder.transform(y_test_reshaped)

In [ ]:
X = df.drop(['Outcome Variable'], axis= 1)
y = df['Outcome Variable']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [ ]:
def compare_algorithms(X_train, y_train, algorithms, cv=5, scoring='accuracy'):
    results = {}

    for name, algorithm in algorithms.items():
        # Perform cross-validation
        scores = cross_val_score(algorithm, X_train, y_train, cv=cv, scoring=scoring)

        # Store the mean and standard deviation of the scores
        results[name] = {
            'mean_accuracy': np.mean(scores),
            'std_accuracy': np.std(scores)
        }

        # Print the results for each algorithm
        print(f"{name} - Mean Accuracy: {results[name]['mean_accuracy']:.4f} | Std Accuracy: {results[name]['std_accuracy']:.4f}")

    return results

In [ ]:
def plot_accuracies(accuracies):
    """
    Plots the mean accuracies and their standard deviations for different algorithms.

    Parameters:
    accuracies (dict): A dictionary where keys are algorithm names and values are dictionaries
                       containing 'mean_accuracy' and 'std_accuracy'.
    """
    # Extracting algorithm names, mean accuracies, and standard deviations
    names = list(accuracies.keys())
    mean_accuracies = [accuracies[name]['mean_accuracy'] for name in names]
    std_accuracies = [accuracies[name]['std_accuracy'] for name in names]

    # Creating the bar plot
    plt.figure(figsize=(10, 6))
    plt.bar(names, mean_accuracies, yerr=std_accuracies, capsize=5, color='skyblue', alpha=0.7)

    # Adding plot details
    plt.xlabel('Algorithms')
    plt.ylabel('Mean Accuracy')
    plt.title('Algorithm Comparison: Mean Accuracy with Standard Deviation')
    plt.ylim(0, 1)  # Accuracy ranges from 0 to 1
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Display the plot
    plt.tight_layout()
    plt.show()

In [ ]:
df
X_test

In [ ]:
# importing libraries
from sklearn.linear_model import LogisticRegression
lreg = LogisticRegression()
lreg.fit(X_train_transformed,y_train)

In [ ]:
from sklearn.metrics import accuracy_score
# predicting values and checking accuracy
lpred = lreg.predict(x_test_transformed)
accuracy_score(lpred , y_test)

In [ ]:
#importing libraries
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#implementing KNN
knn = KNeighborsClassifier()
knn.fit(X_train_transformed,y_train)

In [ ]:
#predicting values and testing accuracy
kpred = knn.predict(x_test_transformed)
accuracy_score(kpred,y_test)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [ ]:
# Define the classifier model
classifier = RandomForestClassifier(n_estimators=100, random_state=0)

# Create the final pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', classifier)
])

# Fit the pipeline on training data
pipeline.fit(X_train, y_train)

# Evaluate the pipeline on test data
accuracy = pipeline.score(X_test, y_test)
print(f'Model accuracy: {accuracy:.2f}')

In [ ]:
from sklearn.model_selection import cross_val_score

# Define the algorithms
algorithms = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=0)
}

# Compare the algorithms
results = compare_algorithms(preprocessor.fit_transform(X_train), y_train, algorithms)

# **Conclusion**

  Random Forest - Mean Accuracy: 0.6526 | Std Accuracy: 0.0472</br>
  Logistic Regression - Mean Accuracy: 0.6486 | Std Accuracy: 0.0857</br>
  K-Nearest Neighbors - Mean Accuracy: 0.6021 | Std Accuracy: 0.0316